In [2]:
import pandas as pd
import numpy as np

In [3]:
video = pd.read_csv('video_data.csv',delimiter = ',')
radar = pd.read_csv('radar_data.csv')
print(radar)

       Unnamed: 0        lat        lon   v_n   v_e   len     id  lane  class  \
0               0  60.160418  24.921638 -8.75  8.55   5.4  204.0   0.0    2.0   
1               1  60.160562  24.922052  8.34  0.45   9.0  209.0   3.0    4.0   
2               2  60.160589  24.921923 -7.95 -0.75   9.0  203.0   1.0    4.0   
3               3  60.160640  24.922041  7.94  1.25  14.0  208.0   3.0    4.0   
4               4  60.160719  24.922022  7.94  1.65   5.6  212.0   3.0    3.0   
...           ...        ...        ...   ...   ...   ...    ...   ...    ...   
52924           9  60.160740  24.921974 -2.05 -0.45   1.6  233.0   2.0    1.0   
52925          10  60.160789  24.921927 -9.05 -2.85   5.4  232.0   1.0    2.0   
52926          11  60.161027  24.921871 -8.95 -2.15   5.6  230.0   1.0    3.0   
52927          12  60.161394  24.921958  7.55  2.05   5.6  229.0   4.0    3.0   
52928          13  60.161393  24.922006  7.55  2.05   5.6  226.0   4.0    3.0   

       cyc_ago  quality    

In [15]:
i = 0 # i kertoo mitä havaintoriviä käydään läpi videodatasta
frvideo = np.unique(video.frame.values)
fr_lenvideo = frvideo.shape[0]
videotstamp = [0]*fr_lenvideo# alustetaan videoaikaleimojen määrä
videotstamp[0] = 0 # tämä videostamps[0] on visuaalisesti itse määritelty 
video_velocities = pd.DataFrame({'lat': [],'lon': [], 'type': [],'frame': [], 'v_n': [],'v_e': [], 'videoID':[]})
while i < (fr_lenvideo//4): # j täytyy olla pienempi kuin radar-havaintojen määärän ja i:n pienempi kuin video-havaintojen
    videodetects = video.loc[(video['frame'] == frvideo[i])&(video['Coordinateslat'] < 60.161)].values
    j = 0
    for row in videodetects:
        if i == 0:
            video_velocities2 = pd.DataFrame({'lat': [videodetects[j][3]],'lon': [videodetects[j][4]], 'type': [videodetects[j][2]],'frame': [videodetects[j][1]], 'v_n': [0],'v_e': [0], 'videoID':[videodetects[j][5]]})
            video_velocities = video_velocities.append(video_velocities2)
        else:
            videodetectsprev = video.loc[(video['frame'] == frvideo[i-1])&(video['Coordinateslat'] < 60.161)].values
            #print(videodetects[j][5])
            if videodetects[j][5] in videodetectsprev:
                result = np.where(videodetectsprev == videodetects[j][5])
                index = result[0][0]
                print(videodetects[j][6],videodetectsprev[index][6],videodetects[j][7],videodetectsprev[index][7],videotstamp[i]-videotstamp[i-1])
                v_e = (videodetects[j][6]-videodetectsprev[index][6])/(videotstamp[i]-videotstamp[i-1])
                v_n = (videodetects[j][7]-videodetectsprev[index][7])/(videotstamp[i]-videotstamp[i-1])
                print(v_n,v_e)
                video_velocities2 = pd.DataFrame({'lat': [videodetects[j][3]],'lon': [videodetects[j][4]], 'type': [videodetects[j][2]],'frame': [videodetects[j][1]], 'v_n': [v_n],'v_e': [v_e], 'videoID':[videodetects[j][5]]})
                video_velocities = video_velocities.append(video_velocities2)
            else:
                video_velocities2 = pd.DataFrame({'lat': [videodetects[j][3]],'lon': [videodetects[j][4]], 'type': [videodetects[j][2]],'frame': [videodetects[j][1]],'v_n': [0],'v_e': [0], 'videoID':[videodetects[j][5]]})
                video_velocities = video_velocities.append(video_velocities2)
        j = j +1
    i = i + 1
    if i < fr_lenvideo:
        videotstamp[i] = videotstamp[i-1]+((frvideo[i]-frvideo[i-1])*(1/30))
print(video_velocities)
video_velocities.to_csv('videowithvelocities.csv')

-2.6445683202287342 -3.0409479159116977 16.812504462163595 16.840661076389058 0.03333333333333333
-0.8446984267638769 11.891387870488904
-2.5173027730843787 -2.6445683202287342 16.803456943007614 16.812504462163595 0.03333333333333333
-0.2714255746794336 3.817966414330667
-2.086255829910044 -2.5173027730843787 16.772787000616955 16.803456943007614 0.03333333333333334
-0.9200982717197801 12.931408295230032
-1.9420484585016689 -2.086255829910044 16.762517377628768 16.772787000616955 0.033333333333333326
-0.3080886896455937 4.326221142251262
-1.893920801973652 -1.9420484585016689 16.759089007510227 16.762517377628768 0.033333333333333326
-0.10285110355624741 1.4438296958405064
-1.3464206278693784 -1.893920801973652 16.72005304085536 16.759089007510227 0.033333333333333326
-1.1710789996460493 16.42500522312821
-1.152280496483979 -1.3464206278693784 16.706195841650036 16.72005304085536 0.033333333333333326
-0.4157159761596675 5.824203941561984
-0.7788513426816791 -1.152280496483979 16.67951

13.735375953320984 13.709182358680325 31.825946377898745 31.761573927059807 0.033333333333333215
1.9311735251681552 0.7858078392197881
14.123323544335086 13.735375953320984 32.414912030346734 31.825946377898745 0.033333333333333215
17.668969573439732 11.638427730423105
14.283883854245627 14.123323544335086 32.623587565779594 32.414912030346734 0.033333333333333215
6.260266062985829 4.8168092973162215
14.563467007396536 14.283883854245627 33.207573817436604 32.623587565779594 0.033333333333333215
17.519587549710344 8.387494594527313
14.687467080896337 14.563467007396536 33.41424217246136 33.207573817436604 0.033333333333333215
6.200050650742619 3.7200022049940307
14.71325030147758 14.687467080896337 33.476247766777355 33.41424217246136 0.033333333333333215
1.860167829479955 0.7734966174372843
14.981025535874412 14.71325030147758 34.026449690438994 33.476247766777355 0.033333333333333215
16.506057709849234 8.033257031905016
15.101212980598824 14.981025535874412 34.220609093994824 34.0264

20.282245210427998 20.17974724666701 53.30661811401354 52.86248898917936 0.033333333333333215
13.323873745025358 3.0749389128296767
3.2297235019634005 3.2464728969076515 66.79167782614635 67.2528192061349 0.033333333333333215
-13.834241399656456 -0.5024818483275313
20.38629085184079 20.282245210427998 53.75770081238951 53.30661811401354 0.033333333333333215
13.532480951279146 3.1213692423837824
3.280360699489305 3.2297235019634005 66.34603896271248 66.79167782614635 0.033333333333333215
-13.369165903016173 1.5191159257771423
20.491920753888703 20.38629085184079 54.21590144829143 53.75770081238951 0.033333333333333215
13.746019077057786 3.1688970614373813
20.59917266167752 20.491920753888703 54.68138960701275 54.21590144829143 0.033333333333333215
13.96464476163947 3.2175572336645546
4.8636963958815835 4.8636963958815835 89.50533637029974 89.50533637029976 0.033333333333333215
-4.2632564145606163e-13 0.0
20.708085528950832 20.59917266167752 55.15434030414573 54.68138960701275 0.03333333

3.3550667871069706 3.195354160747748 93.68064337901122 94.80733320933844 0.033333333333333215
-33.800694909816805 4.791378790776694
2.6830587698118222 2.6904124504009284 53.27031065842756 53.476755982673986 0.033333333333333215
-6.19335972739274 -0.22061041767318457
6.133549792993497 6.143234106820625 75.33238657208459 75.46960826879517 0.033333333333333215
-4.116650901317499 -0.29052941481383704
3.437765231995863 3.3550667871069706 93.27741787060991 93.68064337901122 0.033333333333333215
-12.09676525203922 2.4809533466667855
2.61615747652966 2.6830587698118222 52.8875740328086 53.27031065842756 0.033333333333333215
-11.482098768568859 -2.0070387984648734
6.314446254701392 6.133549792993497 75.70888530512144 75.33238657208459 0.033333333333333215
11.294961991105664 5.426893851236862
3.605124834893625 3.437765231995863 92.61385332756292 93.27741787060991 0.033333333333333215
-19.906936291409707 5.02078808693287
2.6032341308255926 2.61615747652966 52.51446760931037 52.8875740328086 0.033

KeyboardInterrupt: 

In [10]:
video = pd.read_csv('videowithvelocities.csv')
radar = pd.read_csv('radar_data.csv')
avglatvid= np.average(video.v_n)
avglatrad = np.average(radar.v_n)
print(avglatvid)
print(avglatrad)

-0.04476937572271298
1.083379810689792


In [6]:
i = 0 # i kertoo mitä havaintoriviä käydään läpi videodatasta
video = pd.read_csv('videowithvelocities.csv')
frvideo = np.unique(video.frame.values)
fr_lenvideo = frvideo.shape[0]
videotstamp = [0]*fr_lenvideo# alustetaan videoaikaleimojen määrä
videotstamp[0] = 0 # tämä videostamps[0] on visuaalisesti itse määritelty 
video_acceleration = pd.DataFrame({'lat': [],'lon': [], 'type': [],'frame': [], 'v_n': [],'v_e': [], 'a_n':[],'a_e':[],'videoID':[]})
while i < (fr_lenvideo): # j täytyy olla pienempi kuin radar-havaintojen määärän ja i:n pienempi kuin video-havaintojen
    videodetects = video.loc[(video['frame'] == frvideo[i])].values
    j = 0
    for row in videodetects:
        if i <= 1:
            video_acceleration2 = pd.DataFrame({'lat': [videodetects[j][1]],'lon': [videodetects[j][2]], 'type': [videodetects[j][3]],'frame': [videodetects[j][4]], 'v_n': [videodetects[j][5]],'v_e': [videodetects[j][6]],'a_n':[0],'a_e':[0], 'videoID':[videodetects[j][7]]})
            video_acceleration = video_acceleration.append(video_acceleration2)
        else:
            videodetectsprev = video.loc[(video['frame'] == frvideo[i-1])].values
            if videodetects[j][7] in videodetectsprev:
                result = np.where(videodetectsprev == videodetects[j][7])
                index = result[0][0]
                a_n = (videodetects[j][5]-videodetectsprev[index][5])/(videotstamp[i]-videotstamp[i-1])
                a_e = (videodetects[j][6]-videodetectsprev[index][6])/(videotstamp[i]-videotstamp[i-1])
                video_acceleration2 = pd.DataFrame({'lat': [videodetects[j][1]],'lon': [videodetects[j][2]], 'type': [videodetects[j][3]],'frame': [videodetects[j][4]], 'v_n': [videodetects[j][5]],'v_e': [videodetects[j][6]],'a_n':[a_n],'a_e':[a_e], 'videoID':[videodetects[j][7]]})
                video_acceleration = video_acceleration.append(video_acceleration2)
            else:
                video_acceleration2 = pd.DataFrame({'lat': [videodetects[j][1]],'lon': [videodetects[j][2]], 'type': [videodetects[j][3]],'frame': [videodetects[j][4]], 'v_n': [videodetects[j][5]],'v_e': [videodetects[j][6]],'a_n':[0],'a_e':[0], 'videoID':[videodetects[j][7]]})
                video_acceleration = video_acceleration.append(video_acceleration2)
        j = j +1
    i = i + 1
    if i < fr_lenvideo:
        videotstamp[i] = videotstamp[i-1]+((frvideo[i]-frvideo[i-1])*(1/30))
print(video_acceleration)
video_acceleration.to_csv('videowithacceleration.csv')

          lat        lon   type    frame        v_n        v_e         a_n  \
0   60.160238  24.921708    car    102.0   0.000000   0.000000    0.000000   
0   60.160239  24.921714    car    103.0  -0.844698  11.891388    0.000000   
0   60.160240  24.921716    car    104.0  -0.271426   3.817966   17.198186   
0   60.160242  24.921722    car    105.0  -0.920098  12.931408  -19.460181   
0   60.160242  24.921725    car    106.0  -0.308089   4.326221   18.360287   
..        ...        ...    ...      ...        ...        ...         ...   
0   60.160347  24.921919  truck  10870.0  -0.234809   0.656639    7.790032   
0   60.160656  24.922062  truck  10870.0  11.613333   4.564939  349.165763   
0   60.160872  24.921838    car  10870.0 -20.705118   1.322726  -18.011687   
0   60.160247  24.921744    car  10870.0   0.000000   0.000000    1.048864   
0   60.160271  24.921763    car  10870.0  -3.599405   0.553179 -125.614562   

           a_e  videoID  
0     0.000000      1.0  
0     0.000

In [4]:
i = 0 # i kertoo mitä havaintoriviä käydään läpi videodatasta
radar = pd.read_csv('radar_data.csv')
frradar = np.unique(radar.tstamp.values)
fr_lenradar = frradar.shape[0]
radartstamp = [0]*fr_lenradar# alustetaan videoaikaleimojen määrä
radar_acceleration = pd.DataFrame({'lat': [],'lon': [], 'v_n': [],'v_e': [], 'a_n':[],'a_e':[], 'len':[], 'id':[], 'lane':[], 'class':[], 'cyc_ago':[], 'quality':[], 'tstamp':[]})
while i < (fr_lenradar): # j täytyy olla pienempi kuin radar-havaintojen määärän ja i:n pienempi kuin video-havaintojen
    radardetects = radar.loc[(radar['tstamp'] == frradar[i])].values
    j = 0
    for row in radardetects:
        if i <= 1:
            radar_acceleration2 = pd.DataFrame({'lat': [radardetects[j][1]],'lon': [radardetects[j][2]], 'v_n': [radardetects[j][3]],'v_e': [radardetects[j][4]], 'a_n':[0],'a_e':[0], 'len':[radardetects[j][5]], 'id':[radardetects[j][6]], 'lane':[radardetects[j][7]], 'class':[radardetects[j][8]], 'cyc_ago':[radardetects[j][9]], 'quality':[radardetects[j][10]], 'tstamp':[radardetects[j][11]]})
            radar_acceleration = radar_acceleration.append(radar_acceleration2)
        else:
            radardetectsprev = radar.loc[(radar['tstamp'] == frradar[i-1])].values
            if radardetects[j][6] in radardetectsprev:
                result = np.where(radardetectsprev == radardetects[j][6])
                index = result[0][0]
                a_n = (radardetects[j][2]-radardetectsprev[index][2])/(1000*(radardetects[j][11]-radardetectsprev[index][11]))
                a_e = (radardetects[j][3]-radardetectsprev[index][3])/(1000*(radardetects[j][11]-radardetectsprev[index][11]))
                if radardetects[j][5] > 4:
                    radar_acceleration2 = pd.DataFrame({'lat': [radardetects[j][1]],'lon': [radardetects[j][2]], 'v_n': [radardetects[j][3]],'v_e': [radardetects[j][4]], 'a_n':[a_n],'a_e':[a_e], 'len':[radardetects[j][5]], 'id':[radardetects[j][6]], 'lane':[radardetects[j][7]], 'class':[radardetects[j][8]], 'cyc_ago':[radardetects[j][9]], 'quality':[radardetects[j][10]], 'tstamp':[radardetects[j][11]]})
                    radar_acceleration = radar_acceleration.append(radar_acceleration2)
            else:
                if radardetects[j][5] > 4:
                    radar_acceleration2 = pd.DataFrame({'lat': [radardetects[j][1]],'lon': [radardetects[j][2]], 'v_n': [radardetects[j][3]],'v_e': [radardetects[j][4]], 'a_n':[0],'a_e':[0], 'len':[radardetects[j][5]], 'id':[radardetects[j][6]], 'lane':[radardetects[j][7]], 'class':[radardetects[j][8]], 'cyc_ago':[radardetects[j][9]], 'quality':[radardetects[j][10]], 'tstamp':[radardetects[j][11]]})
                    radar_acceleration = radar_acceleration.append(radar_acceleration2)
        j = j +1
    i = i + 1
print(radar_acceleration)
radar_acceleration.to_csv('radarwithacceleration.csv')

          lat        lon   v_n   v_e           a_n       a_e   len     id  \
0   60.160418  24.921638 -8.75  8.55  0.000000e+00  0.000000   5.4  204.0   
0   60.160562  24.922052  8.34  0.45  0.000000e+00  0.000000   9.0  209.0   
0   60.160589  24.921923 -7.95 -0.75  0.000000e+00  0.000000   9.0  203.0   
0   60.160640  24.922041  7.94  1.25  0.000000e+00  0.000000  14.0  208.0   
0   60.160719  24.922022  7.94  1.65  0.000000e+00  0.000000   5.6  212.0   
..        ...        ...   ...   ...           ...       ...   ...    ...   
0   60.160676  24.921937 -8.05 -1.95  1.313112e-11  0.000002   4.6  228.0   
0   60.160789  24.921927 -9.05 -2.85 -7.806519e-11  0.000000   5.4  232.0   
0   60.161027  24.921871 -8.95 -2.15  3.715372e-11  0.000002   5.6  230.0   
0   60.161394  24.921958  7.55  2.05 -1.256748e-11  0.000000   5.6  229.0   
0   60.161393  24.922006  7.55  2.05 -1.313169e-11  0.000000   5.6  226.0   

    lane  class  cyc_ago  quality        tstamp  
0    0.0    2.0      0.0 